Demo code used in my presentation/demo for the AI group on 22/3/2024

In [ ]:
%pip install mtcnn

In [ ]:
import tensorflow as tf
import numpy as np
import glob
from PIL import Image
from mtcnn import MTCNN

In [ ]:
def crop_image(image, box):
    x, y, width, height = box[0], box[1], box[2], box[3]
    return image[y:y+height, x:x+width]

In [ ]:
def resize_rgb_image(image, size=(48, 48)):
    return np.expand_dims(np.array((Image.fromarray(image).convert('L')).resize(size)), -1)

In [ ]:
""" 
I’ve already done the step where we extract the frames from the video, so we’re going to be starting with a folder that has all of those frames in it (the Frames folder below).
Here we’re getting the paths to all of the frames from the video and just storing them.
"""

file_paths = glob.glob("/dbfs/FileStore/tables/Frames/*.jpg")

In [ ]:
""" 
Here we can see some of the different frames from the video. 
"""

display(Image.open(file_paths[0]))

In [ ]:
""" 
Then for each frame, take the pixels and store those in a list.
"""

# Store the pixels for each frame in a list
pixels = []

for file_path in file_paths:
    pixels.append(np.asarray(Image.open(file_path)))

In [ ]:
""" 
Then once we have all those pixels we can move on to the MTCNN step. 
For each image we’ll detect the face. Then if exactly 1 face was detected, crop the image, resize it, convert to grayscale (since the model was trained on grayscale images), and then save the pixels of the cropped image. 
I chose to discard images where more than one face was detected because at this point there is no way to determine which face the image should be cropped to without going in and manually checking it myself 
since I haven’t implemented any sort of remembering previous faces and matching them to each other.
"""

# Use MTCNN to detect faces and then crop the images
detector = MTCNN()
cropped_images = []

for img in pixels:
    output = detector.detect_faces(img)
    if (len(output) == 1):
        resized_image = resize_rgb_image(crop_image(img, output[0]["box"]))
        cropped_images.append(resized_image)

In [ ]:
""" 
Standardizing the data the same way as was done to the training data and then turning into a format that can be passed to the model.
"""

x = cropped_images
x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

x_test = tf.convert_to_tensor(x, np.float32)

In [ ]:
""" 
Then we’ll load in the model and predict on the frames.
"""

# Load in the pretrained model
model = tf.keras.models.load_model('/dbfs/FileStore/tables/Models/v8')

# Predict on all of the frames
probabilities = model.predict(x_test)

In [ ]:
"""
What we actually get back as a prediction for each frame is a list of probabilities, one for each of the facial expression classes in this order: [Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral]
So, for frame 0 this is the output here. 
The probability at index X is the probability that the image belongs/should be classified as class X
"""

probabilities[0]

In [ ]:
""" 
Turn all of the lists of probabilities into an integer representing the class that it’s predicting. 
That class will be the one that had the highest probability.
"""

# For each probability distribution, determine what the prediction is
predicted = []

for pred in probabilities:
    predicted.append(np.argmax(pred))

In [ ]:
"""
Then to get the predicted emotion for the whole video, I took that to be the class that was predicted the most over all of the frames.
"""

# Find most frequent prediction
expressions = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"} # map integers to the corresponding emotion
print(f"Predicted emotion: {expressions[max(set(predicted), key=predicted.count)]}")

In [ ]:
"""
We’re saying that somebody is stressed if over 2/3 of the frames from the video fall into the stress-related classes. 
So down here we’re counting how many of the frames fall into those classes and how many frames we have total, and using that to make the prediction of whether the person is stressed or not.
"""

# Predict if stressed or not
num_stressed = 0
total = 0
for i in range(len(predicted)):
    if predicted[i] in {0, 2, 4}:
        num_stressed += 1
    total += 1

if num_stressed > 2*total/3:
    print("Prediction: Stressed")
else:
    print("Prediction: Not stressed")